In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 16.0 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=169b30145166a2a50cc70d426d88fdcc4b7e7c349558bc6deb7d4009fc7ca064
  Stored in directory: /root/.cache/pip/wheels/5a/54/9b/a89cac960efb57c4c35d41cc7c9f7b80daa21108bc376339b7
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/02 18:33:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark.version

'3.3.2'

In [5]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True)
])

In [6]:
from pyspark import SparkFiles

f = "https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhvhv/fhvhv_tripdata_2021-06.csv.gz"
spark.sparkContext.addFile(f)

In [7]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv("file://"+SparkFiles.get("fhvhv_tripdata_2021-06.csv.gz"))

df.count()

14961892

In [8]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



**Q3**: How many taxi trips were there on June 15?

In [9]:
from pyspark.sql import functions as F

In [10]:
df.withColumn('pickup_date', F.to_date(df.pickup_datetime))\
.filter("pickup_date = '2021-06-15'").count()

452470

In [11]:
df.registerTempTable('fhvhv_trips')

/opt/conda/lib/python3.7/site-packages/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [12]:
spark.sql("""
SELECT
    COUNT(1)
FROM 
    fhvhv_trips
WHERE
    to_date(pickup_datetime) = '2021-06-15';
""").show()

+--------+
|count(1)|
+--------+
|  452470|
+--------+



**Q4**: Longest trip for each day

In [13]:
df.columns

['dispatching_base_num',
 'pickup_datetime',
 'dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'SR_Flag',
 'Affiliated_base_number']

In [14]:
df \
    .withColumn('duration', df.dropoff_datetime.cast('long') - df.pickup_datetime.cast('long')) \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .groupBy('pickup_date') \
        .max('duration') \
    .orderBy('max(duration)', ascending=False) \
    .limit(5) \
    .show()

+-----------+-------------+
|pickup_date|max(duration)|
+-----------+-------------+
| 2021-06-25|       240764|
| 2021-06-22|        91979|
| 2021-06-27|        71931|
| 2021-06-26|        65510|
| 2021-06-23|        59281|
+-----------+-------------+



In [15]:
spark.sql("""
SELECT
    to_date(pickup_datetime) AS pickup_date,
    MAX((CAST(dropoff_datetime AS LONG) - CAST(pickup_datetime AS LONG)) / 3600) AS duration
FROM 
    fhvhv_trips
GROUP BY
    1
ORDER BY
    2 DESC
LIMIT 10;
""").show()

+-----------+------------------+
|pickup_date|          duration|
+-----------+------------------+
| 2021-06-25|  66.8788888888889|
| 2021-06-22|25.549722222222222|
| 2021-06-27|19.980833333333333|
| 2021-06-26|18.197222222222223|
| 2021-06-23|16.466944444444444|
| 2021-06-24|13.909722222222221|
| 2021-06-04|             11.67|
| 2021-06-20|10.984444444444444|
| 2021-06-01|           10.2675|
| 2021-06-28| 9.966388888888888|
+-----------+------------------+



**Q6**: Most common pikcup zone name

In [16]:
f = "https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv"
spark.sparkContext.addFile(f)

In [17]:
df_zones = spark.read.option("header","true").csv("file://"+SparkFiles.get("taxi+_zone_lookup.csv"))

In [18]:
df_zones.count(), df_zones.columns

(265, ['LocationID', 'Borough', 'Zone', 'service_zone'])

In [19]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [20]:
df_zones.registerTempTable('zones')

In [21]:
spark.sql("""
SELECT
    pul.Zone AS pickup_zone_desc,
    COUNT(1)
FROM 
    fhvhv_trips fhv 
    LEFT JOIN zones pul ON fhv.PULocationID = pul.LocationID
GROUP BY 
    1
ORDER BY
    2 DESC
LIMIT 10;
""").show()

+--------------------+--------+
|    pickup_zone_desc|count(1)|
+--------------------+--------+
| Crown Heights North|  231279|
|        East Village|  221244|
|         JFK Airport|  188867|
|      Bushwick South|  187929|
|       East New York|  186780|
|TriBeCa/Civic Center|  164344|
|   LaGuardia Airport|  161596|
|            Union Sq|  158937|
|        West Village|  154698|
|             Astoria|  152493|
+--------------------+--------+



In [22]:
spark.sql("""
SELECT
    CONCAT(pul.Zone, ' / ', dol.Zone) AS pu_do_pair,
    COUNT(1)
FROM 
    fhvhv_trips fhv 
    LEFT JOIN zones pul ON fhv.PULocationID = pul.LocationID
    LEFT JOIN zones dol ON fhv.DOLocationID = dol.LocationID
GROUP BY 
    1
ORDER BY
    2 DESC
LIMIT 5;
""").show()

+--------------------+--------+
|          pu_do_pair|count(1)|
+--------------------+--------+
|East New York / E...|   47926|
|    JFK Airport / NA|   31321|
| Canarsie / Canarsie|   28230|
|Crown Heights Nor...|   25216|
|Borough Park / Bo...|   24778|
+--------------------+--------+



In [23]:
pq_path = 'pq/fhvhv/2021/06/'
df.repartition(12).write.parquet(pq_path)

In [24]:
!ls -lh pq/fhvhv/2021/06

total 284M
-rw-r--r-- 1 root root   0 Mar  2 18:38 _SUCCESS
-rw-r--r-- 1 root root 24M Mar  2 18:38 part-00000-8b0a51f5-00a9-4c9b-b0d7-8efce094fd6f-c000.snappy.parquet
-rw-r--r-- 1 root root 24M Mar  2 18:38 part-00001-8b0a51f5-00a9-4c9b-b0d7-8efce094fd6f-c000.snappy.parquet
-rw-r--r-- 1 root root 24M Mar  2 18:38 part-00002-8b0a51f5-00a9-4c9b-b0d7-8efce094fd6f-c000.snappy.parquet
-rw-r--r-- 1 root root 24M Mar  2 18:38 part-00003-8b0a51f5-00a9-4c9b-b0d7-8efce094fd6f-c000.snappy.parquet
-rw-r--r-- 1 root root 24M Mar  2 18:38 part-00004-8b0a51f5-00a9-4c9b-b0d7-8efce094fd6f-c000.snappy.parquet
-rw-r--r-- 1 root root 24M Mar  2 18:38 part-00005-8b0a51f5-00a9-4c9b-b0d7-8efce094fd6f-c000.snappy.parquet
-rw-r--r-- 1 root root 24M Mar  2 18:38 part-00006-8b0a51f5-00a9-4c9b-b0d7-8efce094fd6f-c000.snappy.parquet
-rw-r--r-- 1 root root 24M Mar  2 18:38 part-00007-8b0a51f5-00a9-4c9b-b0d7-8efce094fd6f-c000.snappy.parquet
-rw-r--r-- 1 root root 24M Mar  2 18:38 part-00008-8b0a51f5-00a9-4c9b-b0d7-8

In [25]:
# df = spark.read.parquet(pq_path)